In [1]:
# parameters
schema = 'Sales'
table = 'InvoiceLines'

In [2]:
# Parameters
schema = "Sales"
table = "OrderLines"


In [3]:
from javelin.source import SqlServerConnection, SqlServerSource
from javelin.secret import VaultInjectorSecretProvider
from javelin.operators import avro_encoder, json_encoder, parquet_encoder, add_hash
from javelin.sink import AzureDataLakeStorageSink

In [4]:
dbcreds = VaultInjectorSecretProvider(filename='core-creds.json')
tenant = VaultInjectorSecretProvider(filename='azure-tenant.json')
jhrole = VaultInjectorSecretProvider(filename='jupyterhub-role.json')

In [5]:
db = SqlServerConnection(
    username=dbcreds.secrets["username"],
    password=dbcreds.secrets["password"],
    host="core.komprest.net",
    port=1433,
    database="WideWorldImporters",
    driver_options={
        "driver": "ODBC Driver 17 for SQL Server",
        "authentication": "SqlPassword",
        "TrustServerCertificate":"yes"
    }
)

In [6]:
src = SqlServerSource(config=db)
src.schema_name=schema
src.table_name=table

In [7]:
adls = AzureDataLakeStorageSink(
    tenant_id=tenant.secrets["tenant_id"],
    client_id=jhrole.secrets["client_id"],
    client_secret=jhrole.secrets["client_secret"],
    storage_acct="cmbadls2",
    container="ingest"
)

flow = src.observable_per(rows=100000).pipe(
    add_hash(),
    avro_encoder()
).subscribe(lambda i: adls.write(i))
flow.dispose()

/home/hub/javelin/javelin/source/sql_server_source.py:69: SAWarning: Did not recognize type 'geography' of column 'DeliveryLocation'
  self._table = Table(self.table_name, _md, autoload_with=_engine)
/home/hub/javelin/javelin/source/sql_server_source.py:69: SAWarning: Did not recognize type 'geography' of column 'Location'
  self._table = Table(self.table_name, _md, autoload_with=_engine)
/home/hub/javelin/javelin/source/sql_server_source.py:69: SAWarning: Did not recognize type 'geography' of column 'Border'
  self._table = Table(self.table_name, _md, autoload_with=_engine)
